In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, first, coalesce

# Configurações do Spark (MANTIDAS)
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

spark = SparkSession.builder.config(conf=conf).appName("UnificarUPA").getOrCreate()

# ======================
# ARQUIVOS E CAMINHOS (CORREÇÃO DE CAMINHO: s3a:// e .csv)
# ======================

s3_trusted_path = 's3a://bucket-trusted-upa-connect-sofh/' 
caminho_sensores = s3_trusted_path + 'tabela_sensores_tratada.csv'
caminho_atendimentos = s3_trusted_path + 'tabela_atendimentos_tratada.csv'
caminho_upa = s3_trusted_path + 'upa.csv'
caminho_paciente = s3_trusted_path + 'paciente.csv'

# ======================
# LEITURA DAS TABELAS (CORREÇÃO AQUI: delimiter=';')
# ======================

# Tabela Sensores
sensores_df = spark.read.option('header', 'true').option('delimiter', ';').csv(caminho_sensores)

# Tabela Atendimentos (assumindo que usa o mesmo delimitador)
atendimento_df = spark.read.option('header', 'true').option('delimiter', ';').option('inferSchema', 'true').csv(caminho_atendimentos)

# Tabela UPA (assumindo que usa o mesmo delimitador)
upa_df = spark.read.option('header', 'true').option('delimiter', ',').csv(caminho_upa)

# Tabela Paciente (assumindo que usa o mesmo delimitador)
paciente_df = spark.read.option('header', 'true').option('delimiter', ',').csv(caminho_paciente)


# ===============================================
# 1. TRATAMENTO/PIVOTAGEM DA TABELA SENSORES
# (O código abaixo já está funcional com a correção anterior)
# ===============================================

sensores_pivot_df = (
    sensores_df
    .withColumn('chave_pivot',
                when(col('fk_sensor') == 1, lit('camera_visao_computacional'))
                .when((col('fk_sensor') == 2) & (col('fk_unid_medida') == 1), lit('temperatura_ambiente'))
                .when((col('fk_sensor') == 2) & (col('fk_unid_medida') == 2), lit('umidade_ambiente'))
                .otherwise(lit('outro_sensor'))
               )
    .groupBy('data_hora', 'fk_upa')
    .pivot('chave_pivot')
    .agg(first('valor'))
)

campos_pivot = ['camera_visao_computacional', 'temperatura_ambiente', 'umidade_ambiente']
for campo in campos_pivot:
    if campo not in sensores_pivot_df.columns:
        sensores_pivot_df = sensores_pivot_df.withColumn(campo, lit(None))

sensores_final_df = sensores_pivot_df.drop('outro_sensor')

# ===============================================
# 2. JUNÇÕES (JOINs)
# ===============================================

# 2.1. Junção Sensores (FULL OUTER) Atendimentos
sensores_renomeado = sensores_final_df.withColumnRenamed('fk_upa', 'fk_upa_sensores')
atendimentos_renomeado = atendimento_df.withColumnRenamed('fk_upa', 'fk_upa_atendimentos')

df_juncao1 = sensores_renomeado.join(
    atendimentos_renomeado,
    on=sensores_renomeado.data_hora == atendimentos_renomeado.data_hora,
    how='full_outer'
).withColumn('data_hora_final', coalesce(sensores_renomeado.data_hora, atendimentos_renomeado.data_hora)) 

df_juncao1 = df_juncao1.withColumn('fk_upa_final', coalesce(col('fk_upa_sensores'), col('fk_upa_atendimentos')))

# Remove colunas duplicadas/intermediárias, exceto a FK consolidada
df_juncao1 = df_juncao1.drop(sensores_renomeado.data_hora).drop(atendimentos_renomeado.data_hora).drop('fk_upa_sensores').drop('fk_upa_atendimentos')
df_juncao1 = df_juncao1.withColumnRenamed('data_hora_final', 'data_hora') 

# 2.2. Junção com UPA (LEFT JOIN)
upa_df_renomeado = upa_df.select(
    col('id_upa'),
    col('nome').alias('nome_upa'),
    col('capacidade_atendimento'),
    col('latitude'),
    col('longitude')
)

df_juncao2 = df_juncao1.join(
    upa_df_renomeado,
    on=df_juncao1.fk_upa_final == upa_df_renomeado.id_upa,
    how='left_outer'
) 

# 2.3. Junção com Paciente (LEFT JOIN)
paciente_df_renomeado = paciente_df.select(
    col('id_paciente'),
    col('nome').alias('nome_paciente'),
    col('cpf'),
    col('data_nascimento'),
    col('biometria')
)

df_final = df_juncao2.join(
    paciente_df_renomeado,
    on=df_juncao2.fk_paciente == paciente_df_renomeado.id_paciente,
    how='left_outer'
).drop('fk_paciente') 

# ===============================================
# 3. SELEÇÃO E RENOMEAÇÃO FINAL
# ===============================================

colunas_resultado = [
    col('data_hora'),
    col('id_upa'),
    col('nome_upa'),
    col('capacidade_atendimento'),
    col('latitude'),
    col('longitude'),
    col('camera_visao_computacional'),
    col('temperatura_ambiente'),
    col('umidade_ambiente'),
    col('temperatura_paciente'),
    col('oximetria_paciente'),
    col('id_paciente'),
    col('nome_paciente'),
    col('cpf'),
    col('data_nascimento'),
    col('biometria'),
    col('triagem_horario'),
    col('triagem_sala'),
    col('sala_de_espera').alias('sala_de_espera_horario'),
    col('consultorio_horario'),
    col('consultorio_sala'),
    col('saida').alias('saida_horario'),
]

df_final_selecionado = df_final.select(*colunas_resultado)

df_final_selecionado.printSchema()
df_final_selecionado.show(5, truncate=False)

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b35902be-8f88-4d3e-8547-676dc73ba6c4;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 577ms :: artifacts dl 18ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

root
 |-- data_hora: string (nullable = true)
 |-- id_upa: string (nullable = true)
 |-- nome_upa: string (nullable = true)
 |-- capacidade_atendimento: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- camera_visao_computacional: string (nullable = true)
 |-- temperatura_ambiente: string (nullable = true)
 |-- umidade_ambiente: string (nullable = true)
 |-- temperatura_paciente: double (nullable = true)
 |-- oximetria_paciente: integer (nullable = true)
 |-- id_paciente: string (nullable = true)
 |-- nome_paciente: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- data_nascimento: string (nullable = true)
 |-- biometria: string (nullable = true)
 |-- triagem_horario: timestamp (nullable = true)
 |-- triagem_sala: string (nullable = true)
 |-- sala_de_espera_horario: timestamp (nullable = true)
 |-- consultorio_horario: timestamp (nullable = true)
 |-- consultorio_sala: string (nullable = true)
 |-- saida_horar

+-------------------+------+--------------------------------------+----------------------+-------------------+-------------------+--------------------------+--------------------+----------------+--------------------+------------------+-----------+-------------+----+---------------+---------+---------------+------------+----------------------+-------------------+----------------+-------------+
|data_hora          |id_upa|nome_upa                              |capacidade_atendimento|latitude           |longitude          |camera_visao_computacional|temperatura_ambiente|umidade_ambiente|temperatura_paciente|oximetria_paciente|id_paciente|nome_paciente|cpf |data_nascimento|biometria|triagem_horario|triagem_sala|sala_de_espera_horario|consultorio_horario|consultorio_sala|saida_horario|
+-------------------+------+--------------------------------------+----------------------+-------------------+-------------------+--------------------------+--------------------+----------------+-------------

In [2]:
FINAL_OUTPUT_DIR = "s3a://bucket-client-upa-connect-sofh/"
FINAL_FILENAME = "tabela_unificada_registros.csv"
TEMP_STAGING_DIR = f"{FINAL_OUTPUT_DIR}/_temp_staging_integrated"

# 1. Escreve o resultado no caminho temporário
print(f"\nEscrevendo dados temporariamente em: {TEMP_STAGING_DIR}")

# NOTA: Coalesce(1) para garantir a geração de um único arquivo CSV.
df_final_selecionado.coalesce(1).write \
    .option('delimiter', ';') \
    .option('header', 'true') \
    .option('encoding', 'UTF-8') \
    .mode('overwrite') \
    .csv(TEMP_STAGING_DIR)

# 2. Renomeia o arquivo gerado
try:
    # Acessa a classe 'Path' da JVM através do gateway do Spark
    Path = spark._jvm.org.apache.hadoop.fs.Path
    
    # Acessa a configuração do Hadoop
    hadoop_conf = spark._jsc.hadoopConfiguration()
    
    # Obtém o objeto FileSystem para o caminho temporário
    fs = Path(TEMP_STAGING_DIR).getFileSystem(hadoop_conf)

    # Encontra o arquivo gerado (part-00000-*.csv) dentro do diretório temporário
    list_status = fs.globStatus(Path(TEMP_STAGING_DIR + "/part-00000-*.csv"))

    if list_status:
        # Pega o caminho completo do arquivo gerado
        generated_file_path = list_status[0].getPath()

        # Define o caminho final e o nome específico para o arquivo
        final_output_path = Path(f"{FINAL_OUTPUT_DIR}/{FINAL_FILENAME}")

        # Renomeia (move) o arquivo para o caminho e nome definitivos
        fs.rename(generated_file_path, final_output_path)
        
        # 3. Deleta o diretório temporário (que ficou vazio) e outros arquivos de metadados
        fs.delete(Path(TEMP_STAGING_DIR), True) 
        
        print(f"\n✅ Base integrada salva e renomeada com sucesso para: {final_output_path}")

    else:
        print("\nErro: Não foi possível encontrar o arquivo CSV gerado (part-00000-*.csv) no caminho temporário.")

except Exception as e:
    print(f"\nOcorreu um erro durante a renomeação do arquivo no S3: {e}")

# Encerra a sessão Spark
spark.stop()


Escrevendo dados temporariamente em: s3a://bucket-client-upa-connect-sofh//_temp_staging_integrated


25/10/19 21:40:37 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/10/19 21:40:38 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.



✅ Base integrada salva e renomeada com sucesso para: s3a://bucket-client-upa-connect-sofh/tabela_unificada_registros.csv
